In [ ]:

import numpy as np
import matplotlib.pyplot as plt
from ima.ima.plotting import cart2pol
from gp_ima.ima import C_ima_digamma, C_ima_sample
import GPy
from tueplots import bundles, figsizes

In [ ]:
import sys

%load_ext autoreload
%autoreload 2

sys.path.insert(0, '.')

In [ ]:
from analysis import plot_typography, estimate2uniform, generate_moebius_data

In [ ]:
USETEX = True

In [ ]:
plt.rcParams.update(bundles.neurips2022(usetex=USETEX))
plt.rcParams.update({
    'text.latex.preamble': [r'\usepackage{amsfonts}', # mathbb
                            r'\usepackage{amsmath}'] # boldsymbol
})

In [ ]:
plot_typography(usetex=USETEX, small=12, medium=16, big=20)

In [ ]:

plt.ion(); plt.style.use('seaborn-pastel')
np.random.seed(42)

In [ ]:
NUM_DATA = 500
LATENT_DIM = OBS_DIM = 2

Z, X, c = generate_moebius_data(NUM_DATA, LATENT_DIM, OBS_DIM)

In [ ]:
plt.scatter(Z[:, 0], Z[:, 1], c=c)
plt.scatter(X[:, 0], X[:, 1], c=c)

In [ ]:
NUM_SEEDS = 5
NUM_SAMPLES_C_IMA = 100

cimas = []
zs  =[]
zs_uni =[]

for i in range(NUM_SEEDS):
    kernel = GPy.kern.RBF(2, ARD=False) + GPy.kern.Bias(2)
    m = GPy.models.BayesianGPLVM(np.asarray(X), 2, kernel=kernel, num_inducing=100)
    m.optimize_restarts(3, optimizer='lbfgs')

    cimas.append(np.mean([C_ima_sample(m) for _ in range(NUM_SAMPLES_C_IMA)]))
    zs.append(m.X.mean)
    zs_uni.append(estimate2uniform(zs[-1]))


In [ ]:


_, cest = cart2pol(Zest[:, 0], Zest[:, 1])
plt.scatter(Zest[:, 0], Zest[:, 1], c=cest)

In [ ]:


_, cest = cart2pol(Zest_uni_cima[:, 0], Zest_uni_cima[:, 1])
plt.scatter(Zest_uni_cima[:, 0], Zest_uni_cima[:, 1], c=cest)

In [ ]:
plt.plot(C_IMA)

In [ ]:
Ds = np.logspace(0, 3, 1000).astype(int)
plt.plot(Ds, [C_ima_digamma(max(1, int(np.log(D))), D) for D in Ds])